- Cho 1 file csv gồm các kết quả test của các mẫu test
- Có 382 tiêu chuẩn test, mỗi tiêu chuẩn đều có Upper Limit và Lower Limit
- Trong file csv có các dòng:
  + Dòng 1 là tên khu vực ( 12 cột đầu chứa thông tin cơ bản (cột thứ 3 là ID của mẫu ), các cột tiếp theo chứa thông tin test)
  + Dòng 2 là tiêu đề các trường
  + Dòng 3 là tên hiển thị (bỏ trống)
  + Dòng 4 là PDCA Priority
  + Dòng 5, 6 là upper limit và lower limit
  + Dòng 7 là đơn vị
  + Từ Dòng 8 trở đi là kết quả test
- Delta được quy định với các tiêu chuẩn test (Delta là giá trị hiệu của số đo lớn nhất và số đo nhỏ nhất theo từng tiêu chuẩn test trong từng lot):
  + tên có đuôi là "RSSI -60.0 QPSK [50/0]" thì delta < 0.25
  + tên có đuôi là "Max Power QPSK [12/19]" thì delta < 0.2
- Và 2 thông số đo RSSI, Max Power này standard deviation< 0.2
- Chọn tổ hợp 10 lần test theo mẫu ID, thoả mãn điều kiện là: có std deviation nhỏ nhất và Delta thõa điều kiện ở trên.

In [ ]:
# Gọi thư việc pandas dưới tên pd cho việc xử lý data
import pandas as pd

In [ ]:
# Xin cấp quyền google drive để google collaboration truy cập
drive_root_path = '/content/drive'
from google.colab import drive
drive.mount(drive_root_path)

# Cài đường dẫn file nhập và file xuất
csv_input_path = f'{drive_root_path}/My Drive/Data tests from Huyen_Pham/Rawdata.csv'
csv_output_path = f'{drive_root_path}/My Drive/Data tests from Huyen_Pham/Output.csv'

Mounted at /content/drive


In [ ]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv(csv_input_path, header=None)

# Đặt tiêu đề cột là dòng 2
data.columns = data.iloc[1]

# Lấy test data từ dòng 8 trở đi
test_data = data.iloc[7:,:]

# Các spec test name là cột 13 trở đi
test_columns = data.columns[12:]

# Chuyển data các kết quả test sang dạng số
test_data[test_columns] = test_data[test_columns].apply(pd.to_numeric, errors='coerce')

<ipython-input-3-1bee9a85bb43>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_columns] = test_data[test_columns].apply(pd.to_numeric, errors='coerce')


In [ ]:
# Tạo delta dict cho các tiêu chuẩn test được quy định
delta_dict = {}
for column in test_columns:
    if column.endswith("RSSI -60.0 QPSK [50/0]"):
        delta_dict[column] = 0.25
    elif column.endswith("Max Power QPSK [12/19]"):
        delta_dict[column] = 0.2

In [ ]:

# Hàm để chọn ra 10 tổ hợp test thoả mãn điều kiện trong mỗi nhóm
def select_10test_combination(group):

  # Dataframe mới chứa dữ liệu kết quả test
  _test_data = group[test_columns]
  stddev_dict = {}
  _10test_combinations = []

  # Sắp xếp dữ liệu theo từng tiêu chuẩn test
  sorted_test_data = _test_data.sort_values(by=test_columns.tolist())

  # Số lượng kết quả test cần xem xét
  num_tests = len(sorted_test_data)

  # Duyệt qua từng kết quả test
  for i in range(num_tests - 9):
    # Lấy ra 10 tổ hợp test
    selected_samples = sorted_test_data.iloc[i:i+10]

    # Tính std deviation, nếu std nhỏ nhất >= 0.2 thì loại tổ hợp này
    stddev = selected_samples.std()
    if stddev.min() >= 0.2:
      continue

    # Tính các delta trong các spec cần tính, lưu lại dạng boolean
    deltas = {}
    for column in delta_dict.keys():
      delta = selected_samples[column].max() - selected_samples[column].min()
      deltas[column] = delta < delta_dict[column]

    # Nếu tất cả đều thỏa điều kiện thì lưu tổ hợp này lại
    if all(deltas.values()):
      _10test_combinations.append({"sample_indexes": selected_samples.index, "stddev": stddev.min()})

  # Sắp xếp các tổ hợp theo std deviation nhỏ nhất đến lớn nhất
  sorted_combinations = sorted(_10test_combinations, key=lambda x: x["stddev"])

  # Lấy tổ hợp đầu tiên nếu có
  return sorted_combinations[0] if sorted_combinations else None

In [ ]:
# Nhóm data theo serialnumber
grouped_data = test_data.groupby("SerialNumber")
grouped_data_list = [group for _, group in grouped_data]

In [ ]:
# Giữ lại các dòng 1-7
idx_to_keep = [0,1,2,3,4,5,6]

# Tiến hành chọn tổ hợp và thêm các số dòng để giữ lại
for idx, group in enumerate(grouped_data_list, 1):
  comb = select_10test_combination(group)
  if comb is not None:
    samples = comb.get("sample_indexes", [])
    idx_to_keep.extend(samples)

In [ ]:
# Lọc data theo danh sách và xuất data
selected_data = data.iloc[idx_to_keep]
selected_data.to_csv(csv_output_path, index=False, header=False)